# 3D Cantilever Beam with Distributed Circular Force

## Overview

This code simulates a 3D cantilever beam subjected to a distributed force applied in a circular pattern on one end. It uses nonlinear hyperelastic material modeling and demonstrates both the finite element solution process and 3D visualization techniques.

## Problem Setup

### Geometry and Mesh
- **Beam dimensions**: 20 × 1 × 1 (length × width × height)
- **Mesh**: 20 × 4 × 4 hexahedral elements
- The beam is fixed at the left end (x=0) and loaded at the right end

### Material Properties
- **Model**: Neo-Hookean hyperelastic solid
- **Young's modulus**: 10,000
- **Poisson's ratio**: 0.3
- Linear elements for displacement field ("Lagrange", 1)

## Physics: Nonlinear Elasticity

The code implements a compressible Neo-Hookean material model:

1. **Kinematics**: Defines deformation gradient $F = I + \nabla u$
2. **Strain measures**: Right Cauchy-Green tensor $C = F^T F$ and its invariants
3. **Stored energy**: $\psi = \frac{\mu}{2}(I_1-3) - \mu \ln J + \frac{\lambda}{2}(\ln J)^2$
4. **Stress**: First Piola-Kirchhoff stress $P = \frac{\partial\psi}{\partial F}$

This nonlinear material model accurately captures large deformations that linear elasticity cannot.

## Load Application

The code applies a circular distributed force at the right end of the beam:

- **Location**: Centered at beam's right end
- **Distribution**: Smooth circular pattern with radius 0.8
- **Direction**: Downward in z-direction
- **Magnitude**: Scaled gradually to promote convergence

The force is implemented as a UFL expression with a spatial distribution function that:
1. Identifies points near the right end of the beam
2. Creates a smooth cosine-based distribution within the circular region
3. Applies force vectors in the specified direction

## Solution Process

The solution uses an incremental Newton-Raphson method:

1. **Problem formulation**: Nonlinear variational problem based on the principle of virtual work
2. **Solver configuration**:
   - Newton's method with relaxed tolerances (1e-6)
   - LU direct solver for linear systems
   - Line search for better convergence
   - 50 maximum iterations

3. **Loading strategy**:
   - Force is applied incrementally over 50 steps
   - Each step increases the force magnitude
   - This incremental approach helps manage nonlinearities

## Visualization

The code creates an animated visualization of the beam deformation:

- **Beam**: Shown as a warped mesh colored by displacement magnitude
- **Force**: Visualized with red disk and blue cones
- **Camera**: Positioned to provide a good view of the deformation
- **Output**: Saved as an animated GIF showing progressive deformation

## Key Features

This simulation demonstrates:
- Handling of geometric and material nonlinearities
- Distributed force application in FEA
- Effective visualization of 3D structural behavior
- Solver strategies for nonlinear convergence

The resulting animation shows how the beam gradually deforms under the applied circular force, with the deflection increasing as the load is incrementally applied.

In [4]:
from dolfinx import log, default_scalar_type
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
import pyvista
import numpy as np
import ufl

from mpi4py import MPI
from dolfinx import fem, mesh, plot

# Set PETSc options for better convergence
from petsc4py import PETSc
PETSc.Options().setValue("snes_linesearch_type", "basic")
PETSc.Options().setValue("ksp_type", "preonly")
PETSc.Options().setValue("pc_type", "lu")

# Beam geometry parameters
L = 20.0  # Length
W = 1.0   # Width
H = 1.0   # Height

# Create a 3D beam mesh - coarser mesh for faster convergence
domain = mesh.create_box(MPI.COMM_WORLD, 
                         [[0.0, 0.0, 0.0], [L, W, H]], 
                         [20, 4, 4],  # Coarser mesh
                         mesh.CellType.hexahedron)

# Function space - use linear elements for speed
V = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim,)))

# Define boundary conditions - fixed at left end
def left(x):
    return np.isclose(x[0], 0)

# Mark boundaries
fdim = domain.topology.dim - 1
left_facets = mesh.locate_entities_boundary(domain, fdim, left)
facet_tag = mesh.meshtags(domain, fdim, left_facets, np.full_like(left_facets, 1))

# Fixed left boundary condition
u_bc = np.array((0, 0, 0), dtype=default_scalar_type)
left_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(1))
bcs = [fem.dirichletbc(u_bc, left_dofs, V)]

# Define test and trial functions
v = ufl.TestFunction(V)
u = fem.Function(V, name="Displacement")

# Material properties - BALANCED for convergence and visible deflection
E = default_scalar_type(1.0e4)  # Not too soft, not too stiff
nu = default_scalar_type(0.3)
mu = fem.Constant(domain, E / (2 * (1 + nu)))
lmbda = fem.Constant(domain, E * nu / ((1 + nu) * (1 - 2 * nu)))

# Kinematics
d = len(u)
I = ufl.variable(ufl.Identity(d))
F = ufl.variable(I + ufl.grad(u))
C = ufl.variable(F.T * F)
Ic = ufl.variable(ufl.tr(C))
J = ufl.variable(ufl.det(F))

# Stored strain energy density (compressible neo-Hookean model)
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
# First Piola-Kirchhoff stress
P = ufl.diff(psi, F)

# Define circular distributed force parameters
# Apply to the right end of the beam
circle_center_x = L - 0.01  # Slightly inset from right end
circle_center_y = W/2       # Centered in Y
circle_center_z = H/2       # Centered in Z
circle_radius = 0.8         # Radius of the force distribution
force_magnitude = 200.0     # MODERATE force magnitude for better convergence
force_direction = [0, 0, -1]  # Force in the negative Z direction (downward)

# Distributed force function
def circular_distributed_force(x):
    # Calculate distance from any point to the center of the circle
    dx = x[0] - circle_center_x
    dy = x[1] - circle_center_y
    dz = x[2] - circle_center_z
    
    # Use distance in the YZ plane (perpendicular to beam axis)
    r_squared = dy**2 + dz**2
    r = ufl.sqrt(r_squared)
    
    # Create a smooth distribution function
    inside_circle = ufl.conditional(r <= circle_radius, 
                                   0.5 * (1 + ufl.cos(np.pi * r / circle_radius)), 0.0)
    
    # Restrict force to the beam tip (last 5% of the beam length)
    at_end = ufl.conditional(x[0] > 0.95*L, 1.0, 0.0)
    
    # Scale by force magnitude and direction
    fx = force_magnitude * inside_circle * force_direction[0] * at_end
    fy = force_magnitude * inside_circle * force_direction[1] * at_end
    fz = force_magnitude * inside_circle * force_direction[2] * at_end
    
    return ufl.as_vector([fx, fy, fz])

# Integration measures
metadata = {"quadrature_degree": 2}  # Reduced for better performance
dx = ufl.Measure("dx", domain=domain, metadata=metadata)

# Variational form with internal forces and distributed force
force_expression = circular_distributed_force(ufl.SpatialCoordinate(domain))
F_form = ufl.inner(ufl.grad(v), P) * dx - ufl.inner(v, force_expression) * dx

# Nonlinear problem and solver
problem = NonlinearProblem(F_form, u, bcs)
solver = NewtonSolver(domain.comm, problem)
solver.atol = 1e-6  # Relaxed tolerance
solver.rtol = 1e-6  # Relaxed tolerance
solver.convergence_criterion = "incremental"
solver.max_it = 50  # More iterations allowed

# Visualization setup
pyvista.start_xvfb()
plotter = pyvista.Plotter(window_size=(1200, 800))
plotter.open_gif("3d_beam_deflection.gif", fps=3)

# Create the visualization mesh
topology, cells, geometry = plot.vtk_mesh(V)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

# Prepare displacement field for visualization
values = np.zeros((geometry.shape[0], 3))
function_grid["u"] = values
function_grid.set_active_vectors("u")

# Create warped mesh for deformation visualization
warp_factor = 1.0
warped = function_grid.warp_by_vector("u", factor=warp_factor)
beam_actor = plotter.add_mesh(warped, show_edges=True, opacity=0.8)

# Create a scalar field for displacement magnitude visualization
Vs = fem.functionspace(domain, ("Lagrange", 1))
magnitude = fem.Function(Vs)
us = fem.Expression(ufl.sqrt(sum([u[i]**2 for i in range(len(u))])), 
                   Vs.element.interpolation_points())

# Visualize the circular force area
disk = pyvista.Disc(inner=0, outer=circle_radius, c_res=36)
disk.translate([circle_center_x, circle_center_y, circle_center_z])
disk.rotate_x(90)  # Orient the disk perpendicular to X axis
force_area = plotter.add_mesh(disk, color="red", opacity=0.5)

# Add force visualization with cones
n_cones = 8
for i in range(n_cones):
    # Generate points in a circular pattern
    theta = 2 * np.pi * i / n_cones
    r = circle_radius * 0.6
    
    # Create cone pointing in negative Z direction
    cone = pyvista.Cone(center=[circle_center_x, 
                               circle_center_y + r * np.cos(theta), 
                               circle_center_z + r * np.sin(theta)], 
                       direction=[0, 0, -1], 
                       height=0.6, 
                       radius=0.15)
    
    plotter.add_mesh(cone, color="blue")

# Add a big cone at the center
center_cone = pyvista.Cone(center=[circle_center_x, circle_center_y, circle_center_z],
                          direction=[0, 0, -1],
                          height=0.8,
                          radius=0.25)
plotter.add_mesh(center_cone, color="blue")

# Set optimal camera angle
plotter.camera_position = [(L*1.5, L*0.7, L*0.7),  # Camera position
                          (L/2, W/2, H/2),         # Focus point
                          (0, 0, 1)]               # Up direction

# Time-stepping loop with MORE STEPS for better convergence
log.set_log_level(log.LogLevel.INFO)
max_load = 1.0
num_steps =50  # More steps for smoother loading

for step in range(1, num_steps + 1):
    # Scale the force gradually
    scaling = step * max_load / num_steps
    print(f"Step {step}/{num_steps} - Force scaling: {scaling:.2f}")
    
    # Update the variational form with scaled force
    force_expression = circular_distributed_force(ufl.SpatialCoordinate(domain)) * scaling
    F_form = ufl.inner(ufl.grad(v), P) * dx - ufl.inner(v, force_expression) * dx
    problem = NonlinearProblem(F_form, u, bcs)
    solver = NewtonSolver(domain.comm, problem)
    solver.atol = 1e-6
    solver.rtol = 1e-6
    solver.max_it = 50
    
    # Solve the nonlinear problem
    try:
        num_its, converged = solver.solve(u)
        u.x.scatter_forward()
        
        if converged:
            print(f"  Converged in {num_its} iterations")
        else:
            print(f"  WARNING: Did not converge after {num_its} iterations")
            
    except Exception as e:
        print(f"  Solver error: {e}")
        # Try to continue anyway
        pass
    
    # Update visualization (even if not fully converged)
    function_grid["u"][:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
    magnitude.interpolate(us)
    
    warped_n = function_grid.warp_by_vector("u", factor=warp_factor)
    warped.points[:, :] = warped_n.points
    warped["mag"] = magnitude.x.array
    warped.set_active_scalars("mag")
    
    # Update color range
    max_val = np.max(magnitude.x.array) if magnitude.x.array.size > 0 else 1.0
    plotter.update_scalar_bar_range([0, max_val])
    
    # Write the frame
    plotter.write_frame()
    
    # If this step converged, use it as a starting point for the next step
    # Otherwise keep the last good solution

plotter.close()
print("\nSimulation completed.")

[2025-04-20 15:40:15.781] [info] Extract basic topology: 2560->2560
[2025-04-20 15:40:15.781] [info] Build local dual graph
[2025-04-20 15:40:15.781] [info] Build local part of mesh dual graph (mixed)
[2025-04-20 15:40:15.781] [info] GPS pseudo-diameter:(26) 319-0
[2025-04-20 15:40:15.781] [info] Create topology (single cell type)
[2025-04-20 15:40:15.781] [info] Create topology (generalised)
[2025-04-20 15:40:15.782] [info] Computing communication graph edges (using NBX algorithm). Number of input edges: 1
[2025-04-20 15:40:15.782] [info] Finished graph edge discovery using NBX algorithm. Number of discovered edges 1
[2025-04-20 15:40:15.782] [info] Compute ghost indices
[2025-04-20 15:40:15.782] [info] Computing communication graph edges (using PCX algorithm). Number of input edges: 0
[2025-04-20 15:40:15.782] [info] Finished graph edge discovery using PCX algorithm. Number of discovered edges 0
[2025-04-20 15:40:15.782] [info] Computing communication graph edges (using NBX algorithm